In [1]:
import pandas as pd
import numpy as np
import zipfile
import pysurveycto
import datetime
import pandas as pd
import pytz
from datetime import datetime, timedelta
from io import StringIO
import re
import pywhatkit as kit
import time

In [2]:
server_name = "rtvuganda"
username = "raphael@raisingthevillage.org"
password = "Evaluation2022"

In [3]:
scto = pysurveycto.SurveyCTOObject(server_name, username, password)

In [4]:
form_id = "bhs_2025_rtv"

In [5]:
# form_id = "bhs_pilot_2025_rtv"

In [6]:
my_form_data = scto.get_form_data(form_id, format="json", shape="long")

C:\Users\RTV-LPT1-233\AppData\Local\Programs\Python\Python313\Lib\site-packages\pysurveycto\pysurveycto.py:305: UserWarning: Shape can only be specified when returning data in csv format. Returning data in 'wide' format.
  warnings.warn(


In [7]:
data = pd.DataFrame(my_form_data)

In [8]:
data['SubmissionDate'] = pd.to_datetime(data['SubmissionDate'])

C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\2060759113.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['SubmissionDate'] = pd.to_datetime(data['SubmissionDate'])


In [9]:
yesterday = (datetime.today() - timedelta(days=1)).date()

In [10]:
data[['status', 'consent_1']] = data[['status', 'consent_1']].astype(float)

In [11]:
data = data[
    (data['SubmissionDate'].dt.date == yesterday) &
    ((data['status'] == 1) | (data['status'] == 7)) &
    (data['consent_1'] == 1)
]

In [12]:
data['starttime'] = pd.to_datetime(data['starttime'])

C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\3606276497.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['starttime'] = pd.to_datetime(data['starttime'])


In [13]:
data['endtime']=pd.to_datetime(data['endtime'])

C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\195728126.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['endtime']=pd.to_datetime(data['endtime'])


In [14]:
data['duration'] = pd.to_numeric(data['duration'], errors='coerce') 

In [15]:
columns_to_drop = [
    "survey_start", "intro_start", "text_audit", "caseid", "intronote",
    "pre_region", "pre_project", "pre_vid", "pre_cohort", "pre_cycle",
    "pre_hhid", "treat_status", "stream", "hh_category", "consent_note",
    "consent_2", "contact_keep", "intro_end", "intro_dur", "GPS",
    "hh_demographics_start", "respondent", "respondent_sex", "hhh_sex",
    "hhh_marital_status", "spouse_stays_family", "hhh_religion", "hhh_tribe",
    "hhh_educ_level", "hhh_read_write", "leadership_head", "spouse_fname",
    "spouse_lname", "spouse_age", "spouse_educ_level", "spouse_read_write",
    "leadership_spouse", "contact_no_check", "contact_pri_owner",
    "alt_contact_yn", "alt_contact", "alt_contact_check", "contact_alt_owner",
    "member_note", "tot_hhmembers", "hh_size", "hh_size_married",
    "hh_size_unmarried", "hh_size_adjusted", "hhmem1", "hhmem2", "hhmem3",
    "hhmem4", "hhmem5", "hhmem6", "hhmem7", "hhmem8", "hhmem9", "hhmem10",
    "hhgender1", "hhgender2", "hhgender3", "hhgender4", "hhgender5",
    "hhgender6", "hhgender7", "hhgender8", "hhgender9", "hhgender10",
    "hhage1", "hhage2", "hhage3", "hhage4", "hhage5", "hhage6", "hhage7",
    "hhage8", "hhage9", "hhage10", "gender_list", "gender_f", "children_num",
    "children_num_u5", "children_num_u18", "children_num_abv18",
    "Six_12_attending", "Six_12_not_attending", "Thirteen_18_attending",
    "females_hh_count", "hh_demographics_end", "hh_demographics_dur",
    "Community_relocation", "Community_relocation_area",
    "Community_relocation_reasons_1", "Community_living_period",
    "Community_living", "Community_living_relatives_3",
    "Community_living_satisfication", "Community_activities_organization",
    "Community_relocate", "ppi_indicators_start", "house_rooms",
    "house_sleeping_rooms", "house_average_occupants",
    "Tenure_status_of_the_household_1", "Material_walls", "Material_roof",
    "Material_floor", "Fuel_source_cooking", "Cooking_tech", "Lighting_source",
    "Type_of_Toilet_Facility", "Every_Member_at_least_ONE_Pair_of_Shoes",
    "Bedding_type_adults_4", "Household_consume_rice",
    "Household_consume_tea_spices", "Household_consume_toothpaste",
    "ppi_indicators_end", "ppi_indicators_dur", "access_info_start",
    "main_info_source", "access_info_end", "access_info_dur",
    "access_health_start", "health_status", "non_waterborne_30_days",
    "waterborne_illness_last_30_days", "waterborne_illness_hh_member_num",
    "hh_member_sleep_under_net", "member_medical_treatment",
    "treatment_facility_1", "HH_treatment_expense", "female_member_birth",
    "family_planning", "access_health_end", "access_health_dur",
    "water_access_start", "Main_source_of_water_for_consumption",
    "Average_Water_Consumed_Per_Day", "water_treat_method_1",
    "water_treat_method_num", "primary_Member_Usually_Collects_water",
    "primary_Member_Usually_Collects_water_gender",
    "primary_Member_Usually_Collects_water_age", "water_access_end",
    "access_water_dur", "assets_start", "farm_implements_owned",
    "heavy_duty_machinery_owned", "bicycle_owned", "solar_owned",
    "radios_owned", "phones_owned", "phones_types_1", "assets_end",
    "assets_dur", "expenditure_start", "note_week",
    "Land_size_for_Crop_Agriculture_Acres", "fertility_level",
    "Land_size_for_Grazing_Livestock__Acres", "Land_Not_used__Fallow_LandAcre",
    "Land_Rented_out_for_income_Acres", "Land_Homestead", "size_land_verify",
    "Rented_hired_Land_over_the_last_12_Months",
    "How_many_Acres_did_the_household_Rent_Hire", "purpose_land_rented_1",
    "purpose_land_rented_2", "fertility_level_rented_land",
    "Rent_Paid_Per_Season", "land_sold", "land_transfered", "land_bought",
    "land_own_end", "land_own_dur", "GPS_Middle_Value", "Season1_cropped",
    "Season1_land", "Season2_cropped", "Season2_land", "Improved_seed",
    "sn_1_crops_start", "Season_1_1", "seasonal_crops_planted", "survey_end",
    "survey_dur", "end_gps", "distance_enum", "duration_minutes",
    "instanceID", "instanceName", "formdef_version", "review_quality",
    "review_status", "KEY", "decision_start", "decision_note",
    "decision_Food_crop_Farming", "decision_Cash_crop_Farming",
    "decision_Livestock_raising", "decision_Non_Farming_activities",
    "decision_Wage_salary_employment", "decision_Fishing",
    "decision_Vegetable_farming", "decision_hh", "decision_1", "decision_2",
    "decision_3", "decision_4", "decision_5", "decision_6",
    "household_decision_making", "community_decision_making", "decision_end",
    "decision_dur", "discord_start", "discord_1", "discord_end", "discord_dur",
    "empowerment_end", "empowerment_dur", "norms_start", "note_norms1",
    "note_norms2", "note_norms3", "norms_r7", "norms_c7", "norms_r8",
    "norms_c8", "norms_r9", "norms_c9", "norms_r10", "norms_c10", "norms_r11",
    "norms_c11", "norms_violence", "norms_end", "norms_dur", "female_qn_end",
    "female_qn_dur", "standards_evaluation_start", "note_gbv",
    "gender_based_violence_1", "gender_based_violence_2",
    "gender_based_violence_issues", "note_authorities",
    "education_gender_based_violence", "reporting_gender_based_violence",
    "community_leaders_gender_based_violence",
    "preventing_leaders_gender_based_violence_1",
    "preventing_leaders_gender_based_violence_2", "standards_note",
    "sectiona_start", "latrine_present", "latrine_cover_present",
    "latrine_floor_sealed", "latrine_door_present", "latrine_walls_sealed",
    "tippy_tap_present", "tippy_fill_water", "soap_ash_present", "tippy_img",
    "latrine_envir_safe", "human_excreta_free", "hangline_present",
    "kitchen_present", "kitchen_ventilated", "kitchen_clean",
    "bathroom_present", "bath_drainage_present", "compound_clean",
    "diskrack_present", "double_dishrack", "wash_home_end", "sectiona_dur",
    "weather_information_4",
"weather_information_5",
"weather_information_6",
"fp_method_modern_hh_4",
"household_savings_start",
"household_primary_save_mode",
"save_mode_6",
"saving_amount_bank_f",
"saving_amount_SACCO_f",
"saving_amount_mm_f",
"saving_amount_shopkeeper_f",
"saving_amount_family_friends_f",
"saving_amount_safe_home",
"saving_amount_safe_home_f",
"saving_amount_rtv_vsla_cashround_f",
"saving_amount_non_rtv_vsla_f",
"saving_amount_self_help_f",
"hhhead_spouse_saving_separate",
"savings_total_raw",
"savings_total_formatted",
"note_savings",
"household_savings_end",
"household_savings_dur",
"household_borrowing_start",
"borrowed_past_12_months",
"household_loans_sources",
"household_owned_money",
"household_borrowing_end",
"household_borrowing_dur",
"daily_meals",
"lack_food",
"foodtime_month_6",
"note_hunger",
"lack_money_food",
"lack_money_healthy_foods",
"lack_money_few_kinds_food",
"skip_meal_lack_money",
"eat_less_lack_money",
"hh_ran_out_food_lack_money",
"hungry_no_eat_lack_money",
"no_meal_all_day_lack_money",
"livestock_income_start",
"livestock_present_hh",
"Livestock_list_1",
"Goats_Number_of_Livestock_Sold",
"Goats_sales",
"Goats_Livestock_lost",
"Goats_dead_sale",
"goat_incomes",
"total_livestock_income",
"total_livestock_income_f",
"livestock_shocks_yn",
"livestock_income_end",
"preffered_manure_making_method",
"composts_num",
"composts_standard",
"forms_non_bio",
"composts_free_runoff",
"composts_freq_use",
"composts_near",
"compost_img",
"impr_farm_end",
"impr_farm_dur",
"water_mgt_methods_1",
"water_mgt_methods_2",
"water_mgt_methods_3",
"water_mgt_methods_4",
"water_mgt_methods_num",
"soil_mgt_methods_1",
"soil_mgt_methods_2",
"soil_mgt_methods_3",
"soil_mgt_methods_4",
"soil_mgt_methods_num",
"bbw_mgt",
"bbw_mgt_options_1",
"drying_method_1",
"drying_method_2",
"drying_method_3",
"sorting_grading_done",
"post_harvest_sys_present",
"hh_produce_lq_manure",
"hh_apply_lq_manure_recent_season",
"lq_manure_liters",
"hh_produce_organics",
"organic_pesticides_ingridients_3",
"note_livestock",
"gs_care",
"gs_pen_constr",
"gs_pen_clean",
"gs_droppings_rem_composted",
"sheep_goat_pen_photo",
"kap_livestock_1",
"kap_livestock_3",
"kap_livestock_4",
"kap_livestock_num",
"kap_farm_practices_1",
"kap_farm_practices_2",
"kap_farm_practices_3",
"kap_farm_practices_4",
"kap_farm_practices_5",
"kap_farm_practices_6",
"kap_farm_practices_7",
"kap_farm_practices_8",
"kap_farm_practices_9",
"kap_farm_practices_10",
"kap_farm_practices_num",
"standards_evaluation_end",
"standards_evaluation_dur",
"weather_support_organization_2",
"weather_advisories_5",
"weather_advisories_6",
"weather_information_1",
"weather_information_3",
"weather_information_effectiveness",
"work_casual_num",
"not_home_comment",
"member1_name_first",
"member1_name_second",
"member1_name",
"member1_sex",
"member1_age",
"age_6_12_Attend_Sch1",
"tuition_hh_contr_member1",
"sch_supplies_member1",
"member1_educ_level",
"children_employed",
"Community_living_relatives_1",
"Bedding_type_adults_2",
"Bedding_type_adults_5",
"Bedding_type_children_2",
"Bedding_type_children_3",
"Bedding_type_children_5",
"water_treat_method_99",
"sch_child_miss_water",
"phones_types_2",
"cereals_week_chk",
"consump_food_gift_check",
"Type_of_land_ownership_3",
"land_acreage_bought",
"land_buyingprice",
"land_location",
"household_primary_save_mode_other",
"save_mode_3",
"saving_amount_mm",
"foodtime_month_3",
"gender_based_violence_99",
"preventing_leaders_gender_based_violence_3",
"water_mgt_methods_99",
"member1_sch_dropout",
"member1_sch_dropout_r_1",
"member2_name_first",
"member2_name_second",
"member2_name",
"member2_sex",
"member2_age",
"age_6_12_Attend_Sch2",
"member2_sch_dropout2",
"tuition_hh_contr_member2",
"sch_supplies_member2",
"member2_educ_level",
"treatment_facility_4",
"fp_method_modern_hh_2",
"fp_method_modern_sp_2",
"season_2_cropping_mthd_2",
"season_2_intercropping_land",
"season_2_intercrop_crops_1",
"season_2_intercrop_crops_2",
"govt_assistance_value_chk",
"sn_1_beans_loss_shocks_1",
"sn_1_beans_loss_shocks_2",
"sn_1_beans_loss_shocks_4",
"sn_1_beans_loss_shocks_5",
"sn_1_beans_loss_shocks_6",
"sn_1_beans_loss_shocks_7",
"sn_1_beans_loss_shocks_8",
"sn_1_beans_loss_qty",
"sn_1_sorghum_Yield_remittance_gift",


"sn_1_sorghum_Yield_stored",
"sn_1_sorghum_Yield_spoiled",
"sn_1_sorghum_loss",
"sn_1_sorghum_loss_shocks_1",
"sn_1_sorghum_loss_shocks_6",
"sn_1_sorghum_loss_shocks_7",
"sn_1_sorghum_loss_shocks_8",
"sn_1_irish_potatoes_Total_Yield_consume",
"sn_1_irish_potatoes_Yield_replanted",
"sn_1_irish_potatoes_Yield_payment_fees_labour",
"sn_1_irish_Yield_stored",
"sn_1_irish_potatoes_Yield_spoiled",
"sn_1_irish_potatoes_loss",
"sn_1_irish_potatoes_1_price_check",

"sn_1_irish_potatoes_yield_irish_potatoes",
"sn_1_sweetpotatoes_planted",
"sn_1_sweetpotatoes_planted_impr",
"sn_1_sweetpotatoes_Total_Yield",
"cereals_week",
"cereals_days",
"tubers_week",
"tubers_days",
"pulses_week",
"pulses_days",
"milk_week",
"milk_days",
"vegetables_week",
"vegetables_days",
"fruits_week",
"fruits_days",
"meat_poultry_offals",
"meat_poultry_offals_days",
"eggs",
"eggs_days",
"fish_week",
"fish_days",
"sugar_week",
"sugar_days",
"fat_oils_week",
"fat_oils_days",
"misc_week",
"misc_days",
"snacks_week",
"snacks_days",
"alcohol_week",
"tobacco_week",
"consump_food_gift_week",
"consump_inkind_week",
"expenditure_end",
"expenditure_dur",
"land_own_start",
"Does_your_Household_own_any_Land",
"Type_of_land_ownership_1",
"land_primary_owner"

]
data = data.drop(columns=columns_to_drop, errors="ignore")

In [16]:
data['duration2']=data['duration']/60

In [17]:
data['is_start_time_beyond_8pm'] = (data['starttime'].dt.hour >= 20).astype(int)

In [18]:
data['starttime'] = pd.to_datetime(data['starttime']).dt.date
data['endtime'] = pd.to_datetime(data['endtime']).dt.date
data['date_difference'] = (data['endtime'] - data['starttime']).apply(lambda x: x.days)

In [19]:
data['is_start_time_beyond_8pm'].min(),data['is_start_time_beyond_8pm'].max()

(np.int64(0), np.int64(1))

In [20]:
data['is_duration_invalid']=((data['duration2']<20) | (data['duration2']>60)).astype(int)

In [21]:
data['sample'] = data['enumerator_name'].map(data['enumerator_name'].value_counts())

In [22]:
list_ = ['beans', 'maize','peas','cassava']  
season = [1, 2]
state = ['fresh', 'dry']
price_1 = []

for i in season:
    for j in list_:
        for k in state:
            price1 = f'sn_{i}_{j}_Market_Price_{k}'
            if price1 in data.columns:
                price_1.append(price1)  

print(price_1)

['sn_1_beans_Market_Price_fresh', 'sn_1_beans_Market_Price_dry', 'sn_1_maize_Market_Price_fresh', 'sn_1_maize_Market_Price_dry', 'sn_1_peas_Market_Price_fresh', 'sn_1_peas_Market_Price_dry', 'sn_1_cassava_Market_Price_fresh', 'sn_1_cassava_Market_Price_dry', 'sn_2_beans_Market_Price_fresh', 'sn_2_beans_Market_Price_dry', 'sn_2_maize_Market_Price_fresh', 'sn_2_maize_Market_Price_dry', 'sn_2_peas_Market_Price_fresh', 'sn_2_peas_Market_Price_dry', 'sn_2_cassava_Market_Price_fresh', 'sn_2_cassava_Market_Price_dry']


In [23]:
list=['gnuts','yams','sweetpotatoes','irish_potatoes',
      'ginger','garlic','rice','sorghum','millet',
      'soya_beans']
season=[1,2]
price_2=[]
for i in list:
    for j in season:
        price1=f'sn_{j}_{i}_Market_Price'
        if price1 in data.columns:
            price_2.append(price1)
print(price_2)        

['sn_1_gnuts_Market_Price', 'sn_2_gnuts_Market_Price', 'sn_1_yams_Market_Price', 'sn_2_yams_Market_Price', 'sn_1_sweetpotatoes_Market_Price', 'sn_2_sweetpotatoes_Market_Price', 'sn_1_irish_potatoes_Market_Price', 'sn_2_irish_potatoes_Market_Price', 'sn_1_ginger_Market_Price', 'sn_1_garlic_Market_Price', 'sn_2_garlic_Market_Price', 'sn_1_rice_Market_Price', 'sn_2_rice_Market_Price', 'sn_1_sorghum_Market_Price', 'sn_2_sorghum_Market_Price', 'sn_1_millet_Market_Price', 'sn_2_millet_Market_Price', 'sn_1_soya_beans_Market_Price', 'sn_2_soya_beans_Market_Price']


In [24]:
season_crop_prices=price_1+price_2

In [25]:
for column in season_crop_prices:
    data[column] = pd.to_numeric(data[column], errors='coerce')

In [26]:
price_df=pd.read_excel("crop_prices.xlsx")

In [27]:
veg = [
    "Pumpkins",
    "Carrots",
    "onions",
    "green_pepper",
    "hot_pepper",
    "cabbage",
    "tomato",
    "watermelon",
    "spinach",
    "cauliflower",
    "sukuma_wiki",
    "beetroot",
    "blacknightshade",
    "white_eggplant",
    "green_eggplant",
    "purple_eggplant",
    "dodo"
]
season=[1,2]
veg_list=[]
for k in veg:
    for j in season:
        vegk=f'sn_{j}_{k}_Market_Price'
        if vegk in data.columns:
            veg_list.append(vegk)

In [28]:
crops=['gnuts','beans','irish_potatoes','maize','peas',
      'ginger','garlic','rice','barley','sorghum','millet',
      'soya_beans']
seasons=[1,2]

In [29]:
for crop in crops:
    for season in seasons:
        total_yield = f'sn_{season}_{crop}_Total_Yield'
        planted = f'sn_{season}_{crop}_planted'
        
        if total_yield not in data.columns:
            data[total_yield] = np.nan
        if planted not in data.columns:
            data[planted] = np.nan
        data[total_yield] = pd.to_numeric(data[total_yield], errors='coerce')
        data[planted] = pd.to_numeric(data[planted], errors='coerce')

C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\962021301.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[total_yield] = np.nan
C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\962021301.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[planted] = np.nan
C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\962021301.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

In [30]:
def calculate_yield_per_plant(data, crops, seasons):
    result = data.copy()
    
    for crop in crops:
        for season in seasons:
            total_yield = f'sn_{season}_{crop}_Total_Yield'
            planted = f'sn_{season}_{crop}_planted'
            yield_per_plant = f'sn_{season}_{crop}_yp'
            
            # Ensure the yield_per_plant column exists
            if yield_per_plant not in result.columns:
                result[yield_per_plant] = np.nan

            if total_yield in result.columns and planted in result.columns:
                # Identify rows where both total_yield and planted are missing
                null_mask = result[total_yield].isna() & result[planted].isna()
                result.loc[null_mask, yield_per_plant] = np.nan  # Explicitly set to NaN
                
                # Compute yield per plant where valid values exist
                mask = (
                    pd.notna(result[total_yield]) & 
                    pd.notna(result[planted]) & 
                    (result[planted] != 0)
                )
                
                if mask.any():
                    result.loc[mask, yield_per_plant] = (
                        result.loc[mask, total_yield] / result.loc[mask, planted]
                    )
                    
                    # Replace infinite values with NaN
                    result[yield_per_plant].replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return result
data = calculate_yield_per_plant(data, crops, seasons)


C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\565945013.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result[yield_per_plant].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\565945013.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

In [31]:
print([col for col in data.columns if col.endswith("_yp")])

['sn_1_gnuts_yp', 'sn_2_gnuts_yp', 'sn_1_beans_yp', 'sn_2_beans_yp', 'sn_1_irish_potatoes_yp', 'sn_2_irish_potatoes_yp', 'sn_1_maize_yp', 'sn_2_maize_yp', 'sn_1_peas_yp', 'sn_2_peas_yp', 'sn_1_ginger_yp', 'sn_2_ginger_yp', 'sn_1_garlic_yp', 'sn_2_garlic_yp', 'sn_1_rice_yp', 'sn_2_rice_yp', 'sn_1_barley_yp', 'sn_2_barley_yp', 'sn_1_sorghum_yp', 'sn_2_sorghum_yp', 'sn_1_millet_yp', 'sn_2_millet_yp', 'sn_1_soya_beans_yp', 'sn_2_soya_beans_yp']


In [32]:
required_columns = [
    'Distance_travelled_one_way_OPD_treatment', 'Time_travel_one_way_trip_OPD_treatment_minutes',
    'water_distance_collect_water_round_trip', 'hh_water_collection_Minutes', 'distance_primary_market',
    'time_primary_market', 'Size_land_owned', 'Value_land_owned', "govt_assistance_value", 
    "remittance_gifts_children", "remittance_gifts_relative", "remittance_gifts_friends",
    "casual_work_wage_member1", "casual_work_inkind_member1", "casual_work_wage_member2", 
    "casual_work_inkind_member2", "casual_work_wage_member3", "casual_work_inkind_member3", 
    "casual_work_wage_member4", "casual_work_inkind_member4", "casual_work_wage_member5", 
    "casual_work_inkind_member5", "casual_work_wage_member6", "casual_work_inkind_member6"
]

for col in required_columns:
    if col not in data.columns:
        data[col] = np.nan


data[required_columns] = data[required_columns].astype(float)

In [33]:
col_price = [i for i in data.columns if "price" in i or "payment_fees_labour" in i or "remittance" in i or "market_ppu" in i or "Price" in i]
data[col_price] = data[col_price].apply(pd.to_numeric, errors='coerce')


In [34]:
def highlight_out_of_range(cell_value, min_value, max_value):
    if pd.notna(cell_value) and isinstance(cell_value, (int, float)):  # Ensure it's numeric
        if cell_value < min_value or cell_value > max_value:
            return "background-color: red"
    return ""

def apply_highlighting(data, price_df):
    # Convert price_df to a dictionary for fast lookup (case insensitive)
    crop_prices = {row["Crop"].strip(): (row["Min"], row["Max"]) for _, row in price_df.iterrows()}

    # Apply the highlighting function column-wise
    def highlight_column(column):
        crop_name = column.name.strip()  # Normalize column name
        price_range = crop_prices.get(crop_name)  # Safe dictionary lookup
        if price_range:
            min_price, max_price = price_range
            return column.apply(lambda x: highlight_out_of_range(x, min_price, max_price))
        return [""] * len(column)  # No styling if no matching crop

    return data.style.apply(highlight_column, subset=data.columns)

In [35]:
col = [i for i in data.columns if "Value_remittance_gift" in i or "Value_payment_fees_labour" in i or 'remittance' in i]

def extract_crop(col_name):
    match = re.search(r'sn_\d+_([a-zA-Z_]+)_Yield|sn_\d+_([a-zA-Z_]+)_Value|([a-zA-Z_]+)_Yield|([a-zA-Z_]+)_Value|([a-zA-Z_]+)_remittance|([a-zA-Z_]+)_payment_fees', col_name)
    if match:
        return match.group(1) or match.group(2) or match.group(3) or match.group(4) or match.group(5) or match.group(6)
    return "Unknown"
df = pd.DataFrame({'Column Name': col, 'Crop Name': [extract_crop(coll) for coll in col]})
data[col] = data[col].apply(pd.to_numeric, errors='coerce')

In [36]:
business_number = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,97] 
for bus in business_number:
    bus_sa = f'business{bus}_sales'
    bus_po = f'business{bus}_profit'
    if bus_sa not in data.columns and bus_po not in data.columns:
        data[bus_sa]=np.nan
        data[bus_po]=np.nan
    
    if bus_sa in data.columns and bus_po in data.columns:
        data[[bus_sa,bus_po]]=data[[bus_sa,bus_po]].astype(float)
    else:
        print(f"Columns {bus_sa} or {bus_po} do not exist in the dataset.")

C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\56482326.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[bus_sa]=np.nan
C:\Users\RTV-LPT1-233\AppData\Local\Temp\ipykernel_21256\56482326.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[bus_po]=np.nan


In [37]:
def highlight_time(row):
    colors = [''] * len(row)

    # Highlight remittance and payment fee values
    col = [i for i in data.columns if "Value_remittance_gift" in i or "Value_payment_fees_labour" in i or 'remittance_gifts_friends' in i]
    for col_name in col:
        if col_name in row.index and not pd.isna(row[col_name]):
            if row[col_name] < 100 or row[col_name] % 100 != 0:
                colors[row.index.get_loc(col_name)] = 'background-color:red'

    # Sample Check
    if row['sample'] < 6:
        colors[row.index.get_loc('sample')] = 'background-color:red'

    # OPD Travel Checks
    if row['Distance_travelled_one_way_OPD_treatment'] == 0 or row['Distance_travelled_one_way_OPD_treatment'] >= 28:
        colors[row.index.get_loc('Distance_travelled_one_way_OPD_treatment')] = 'background-color:red'

    if row['Time_travel_one_way_trip_OPD_treatment_minutes'] == 0 or row['Time_travel_one_way_trip_OPD_treatment_minutes'] >= 420:
        colors[row.index.get_loc('Time_travel_one_way_trip_OPD_treatment_minutes')] = 'background-color:red'

    # Water Collection Checks
    if row['water_distance_collect_water_round_trip'] == 0 or row['water_distance_collect_water_round_trip'] > 8:
        colors[row.index.get_loc('water_distance_collect_water_round_trip')] = 'background-color:red'

    if row['hh_water_collection_Minutes'] == 0 or row['hh_water_collection_Minutes'] >= 420:
        colors[row.index.get_loc('hh_water_collection_Minutes')] = 'background-color:red'

    # Market Travel Checks
    if row['distance_primary_market'] == 0 or row['distance_primary_market'] > 10:
        colors[row.index.get_loc('distance_primary_market')] = 'background-color:red'

    if row['time_primary_market'] == 0 or row['time_primary_market'] >= 420:
        colors[row.index.get_loc('time_primary_market')] = 'background-color:red'

    # 🚨 Walking Speed Rule Check (Only for high travel time)
    expected_time_opd = row['Distance_travelled_one_way_OPD_treatment'] * 40
    if row['Time_travel_one_way_trip_OPD_treatment_minutes'] > 1.2 * expected_time_opd:
        colors[row.index.get_loc('Distance_travelled_one_way_OPD_treatment')] = 'background-color:red'
        colors[row.index.get_loc('Time_travel_one_way_trip_OPD_treatment_minutes')] = 'background-color:red'

    expected_time_water = row['water_distance_collect_water_round_trip'] * 60
    if row['hh_water_collection_Minutes'] > 1.2 * expected_time_water:
        colors[row.index.get_loc('water_distance_collect_water_round_trip')] = 'background-color:red'
        colors[row.index.get_loc('hh_water_collection_Minutes')] = 'background-color:red'

    expected_time_market = row['distance_primary_market'] * 40
    if row['time_primary_market'] > 1.2 * expected_time_market:
        colors[row.index.get_loc('distance_primary_market')] = 'background-color:red'
        colors[row.index.get_loc('time_primary_market')] = 'background-color:red'

    # Business Check
    business_number = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,97]
    for bus in business_number:
        bus_sa = f'business{bus}_sales'
        bus_po = f'business{bus}_profit'
        if bus_sa in row.index and bus_po in row.index:
            if row[bus_po] > row[bus_sa]:
                colors[row.index.get_loc(bus_po)] = 'background-color:red'
                colors[row.index.get_loc(bus_sa)] = 'background-color:red'

    # Crop Yield Checks
    crop_yield_thresholds = {
        'sn_1_beans_yp': 100, 'sn_2_beans_yp': 100,
        'sn_1_maize_yp': 250, 'sn_2_maize_yp': 250,
        'sn_1_peas_yp': 150, 'sn_2_peas_yp': 150,
        'sn_1_gnuts_yp': 150, 'sn_2_gnuts_yp': 150,
        'sn_1_irish_potatoes_yp': 10, 'sn_2_irish_potatoes_yp': 10,
        'sn_1_ginger_yp': 100, 'sn_2_ginger_yp': 100,
        'sn_1_rice_yp': 150, 'sn_2_rice_yp': 150,
        'sn_1_barley_yp': 100, 'sn_2_barley_yp': 100,
        'sn_1_sorghum_yp': 100, 'sn_2_sorghum_yp': 100,
        'sn_1_millet_yp': 200, 'sn_2_millet_yp': 200,
        'sn_1_soya_beans_yp': 100, 'sn_2_soya_beans_yp': 100,
        'sn_1_garlic_yp': 100, 'sn_2_garlic_yp': 100
    }

    for crop, threshold in crop_yield_thresholds.items():
        if crop in row.index and row[crop] > threshold:
            colors[row.index.get_loc(crop.replace("_yp", "_planted"))] = 'background-color:red'
            colors[row.index.get_loc(crop.replace("_yp", "_Total_Yield"))] = 'background-color:red'
            colors[row.index.get_loc(crop)] = 'background-color:red'

    # Land Value Check
    if 'Size_land_owned' in row.index and 'Value_land_owned' in row.index:
        expected_land_value = row['Size_land_owned'] * 15000000
        if row['Value_land_owned'] > expected_land_value:
            colors[row.index.get_loc('Size_land_owned')] = 'background-color:red'
            colors[row.index.get_loc('Value_land_owned')] = 'background-color:red'

    # Time Checks
    if 'is_start_time_beyond_8pm' in row.index and row['is_start_time_beyond_8pm'] == 1:
        colors[row.index.get_loc('starttime')] = 'background-color:red'

    if 'is_duration_invalid' in row.index and row['is_duration_invalid'] == 1:
        colors[row.index.get_loc('duration2')] = 'background-color:red'

    # Date Difference Check
    if 'date_difference' in row.index and row['date_difference'] > 0:
        colors[row.index.get_loc('starttime')] = 'background-color:red'
        colors[row.index.get_loc('endtime')] = 'background-color:red'

    return colors

In [38]:
styled=styled_df = apply_highlighting(data, price_df).apply(highlight_time, axis=1)

In [39]:
excel_output="styled_output2.xlsx"

In [40]:
styled.to_excel(excel_output, engine="openpyxl", index=False)

In [41]:
time.sleep(15)

In [42]:
# recipient_number = "+256752537353"
# kit.sendwhats_image(recipient_number, excel_output, "Here is the analysis result.")

In [ ]:
# update content

## 